To view all AI models accessible with your API key

In [ ]:
import google.generativeai as genai

genai.configure(api_key="YOUR API KEY")
models = genai.list_models()
for m in models:
    print(m.name)

To confirm the API key is working, has sufficient quota, and hasn't hit rate limits.

In [ ]:
genai.configure(api_key="YOUR API KEY")
model = genai.GenerativeModel('models/gemini-1.5-flash-latest')

try:
    response = model.generate_content("Say hello!")
    print(response.text)
except Exception as e:
    print("Gemini API error:", e)